In [1]:
import pandas as pd
import numpy as np
import torch
import os

dataset_path = './dataset/'
dataset_files = ['TcData.csv' ,'BsData.csv']

tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}

dfs = dict([])
for file in dataset_files:
    df = pd.read_csv(os.path.join(dataset_path,file), na_values=0)
    df = df.rename(columns={'Composition': 'formula'})
    df.name = file[:2]
    dfs[df.name] = df

In [2]:
from utils import add_feature

load_df = True
for df_name, df in dfs.items():
    if load_df:
        dfs[df_name] = pd.read_pickle(os.path.join(dataset_path, f"./df_data_{df.name}.pkl"))
    else:    
        name = df.name
        df = add_feature.add_composition(df)
        df = add_feature.add_element_fraction(df)
        df.name = name
        dfs[df_name] = df 
        df.to_pickle(os.path.join(dataset_path, f"./df_data_{df.name}.pkl"))

In [3]:
from utils.surrogate import surrogate_model

Tc_surrogate = surrogate_model(name='Tc', df=dfs['Tc'])
Bs_surrogate = surrogate_model(name='Bs', df=dfs['Bs'])

In [4]:
Tc_surrogate.cleanup_df(drop_NaN = False, drop_col_with_NaN = True)
Bs_surrogate.cleanup_df(drop_NaN = False, drop_col_with_NaN = True)

In [5]:
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import ExactMarginalLogLikelihood
from gpytorch.kernels import RBFKernel, ScaleKernel
#from botorch.fit import fit_gpytorch_mll
from torch.optim import SGD, Adam
from botorch.models.transforms.input import ChainedInputTransform
from utils.model import trainGP, NormalizeElementFractions, NormalizeFeatures
from utils.model import test_features_normalized

for surrogate_model in [Tc_surrogate, Bs_surrogate]:
    surrogate_model.model = None
    outcome_transform = Standardize(m=1)
    normalize_other = NormalizeFeatures(indices=surrogate_model.to_scale_col)
    normalize_EF = NormalizeElementFractions(indices=surrogate_model.EF_col)
    input_transform = ChainedInputTransform(
        tf1=normalize_other, 
        tf2=normalize_EF,
    )
    
    surrogate_model.model = SingleTaskGP(
        torch.tensor(surrogate_model.X), 
        torch.tensor(surrogate_model.y).unsqueeze(dim=1),
        input_transform=input_transform,
        outcome_transform=outcome_transform,
        covar_module=ScaleKernel(RBFKernel()),
    )
    
    print("-------------------")

    surrogate_model.model.train()
    #optimizer_kwargs = {'lr': 1e-2, 'weight_decay': 1e-3}
    optimizer_kwargs = {'lr': 5e-2, 'weight_decay': 1e-4}
    print(f"Training {surrogate_model.name}")
    #surrogate_model.model.train()
    trainGP(surrogate_model.model,
            mll = ExactMarginalLogLikelihood(surrogate_model.model.likelihood, surrogate_model.model), 
            optimizer = SGD([{'params': surrogate_model.model.parameters()}], **optimizer_kwargs),
            num_epochs=600,
            print_interval = 100 
           )
    print("-------------------")

torch.Size([220, 104])
-------------------
Training Tc
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
torch.Size([220, 104])
to

In [6]:
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from gpytorch.kernels import RBFKernel, ScaleKernel
#from botorch.fit import fit_gpytorch_mll
from botorch.models.transforms.input import ChainedInputTransform
from utils.model import trainGP, NormalizeElementFractions, NormalizeFeatures
from utils.model import test_features_normalized

def initialize_model(train_x, train_obj):
    
    models = []
    #first index goes over obj_idx
    for i in range(train_obj.shape[-1]):
        y = train_obj[..., i]
        X = train_x
        
        outcome_transform = Standardize(m=1)
        normalize_other = NormalizeFeatures(indices=surrogates[i].to_scale_col)
        normalize_EF = NormalizeElementFractions(indices=surrogates[i].EF_col)
        input_transform = ChainedInputTransform(
            tf1=normalize_other, 
            tf2=normalize_EF,
        )
        
        model = SingleTaskGP(
            X,
            y, #.unsqueeze(dim=1), 
            input_transform=input_transform,
            outcome_transform=outcome_transform,
            covar_module=ScaleKernel(RBFKernel()),
        )
        models.append(model)
    
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return mll, model

In [7]:
#from utils.model import trainGPList
from torch.optim import Adam, SGD
def train_model(modelList: ModelListGP, mllList: SumMarginalLogLikelihood):
    modelList.train()
    #optimizer_kwargs = {'lr': 1e-2, 'weight_decay': 1e-3}
    optimizer_kwargs = {'lr': 3e-2, 'weight_decay': 1e-3}
    print("-------------------")
    for model, mll in zip(modelList.models, mllList.mlls):
    #print(f"Training {model_name}")
    #surrogate_model.model.train()
        trainGP(model,
                mll = mll, 
                optimizer = Adam([{'params': model.parameters()}], **optimizer_kwargs),
                num_epochs=50,
                print_interval = 100 
               )
        print("-------------------")

In [8]:
SMOKE_TEST=True
surrogates = [Tc_surrogate, Bs_surrogate]
ref_point = torch.tensor([np.average(surrogate.y) for surrogate in surrogates],**tkwargs)

In [9]:
def generate_initial_data(surrogates):
    for surrogate in surrogates:
        try:
            surrogate.df = surrogate.df.drop(['formula', 'composition', surrogate.label],axis=1)
        except KeyError:
            pass
        

    total_df = pd.concat([surrogate.df for surrogate in surrogates])
    empty_col = total_df.columns[[not total_df[col].any() for col in total_df.columns]]
    total_df = total_df.drop(empty_col,axis=1)

    
    
    from utils.model import evaluateGP
    X = np.concatenate([surrogate.X for surrogate in surrogates],axis=0)
    y = [evaluateGP(surrogate.model, X) for surrogate in surrogates][0]
    X = torch.tensor(X, **tkwargs)
    y = [torch.tensor(array, **tkwargs) for array in y]
    #X = torch.stack((X, X),dim=-1)
    y = torch.stack(y,dim=-1).unsqueeze(dim=1)
    
    return X, y, total_df

In [34]:
from botorch.optim.optimize import optimize_acqf, optimize_acqf_list
from botorch.acquisition.objective import GenericMCObjective
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
from botorch.utils.multi_objective.box_decompositions.non_dominated import (
    FastNondominatedPartitioning,
)
from botorch.acquisition.multi_objective.monte_carlo import (
    qExpectedHypervolumeImprovement,
    qNoisyExpectedHypervolumeImprovement,
)
from botorch.acquisition.multi_objective.logei import qLogExpectedHypervolumeImprovement
from botorch.utils.sampling import sample_simplex
from botorch.utils.transforms import unnormalize, normalize


BATCH_SIZE = 1
NUM_RESTARTS = 10 if not SMOKE_TEST else 2
RAW_SAMPLES = 512 if not SMOKE_TEST else 4

standard_bounds = torch.zeros(2, 2, **tkwargs)
standard_bounds[1] = 1
bounds = torch.zeros(2, 104, **tkwargs)
bounds[1,:] = 1



def optimize_qehvi_and_get_observation(model, train_x, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    with torch.no_grad():
        pred = model.posterior(normalize(train_x, bounds)).mean
        #pred = torch.concatenate([model.posterior(train_x[:,:,i]).mean for i, model in enumerate(model_qparego.models)],dim=-1)
    partitioning = FastNondominatedPartitioning(
        ref_point=ref_point,
        Y=pred,
    )
    acq_func = qExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point,
        partitioning=partitioning,
        sampler=sampler,
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
        sequential=False,
        return_best_only=False,
    )
    # observe new values
    new_x = unnormalize(candidates.detach(), bounds=bounds)
    print(new_x.shape)
    new_obj_true = torch.stack([model.models[0](new_x).mean, model.models[1](new_x).mean], dim=-1).unsqueeze(1)
    #new_obj = new_obj_true + torch.randn_like(new_obj_true) * NOISE_SE
    return new_x, new_obj_true

In [35]:
def optimize_qnehvi_and_get_observation(model, train_x, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    acq_func = qNoisyExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point.tolist(),  # use known reference point
        X_baseline=normalize(train_x, bounds),
        prune_baseline=True,  # prune baseline points that have estimated zero probability of being Pareto optimal
        sampler=sampler,
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 10, "maxiter": 200},
        sequential=True,
    )
    # observe new values
    new_x = unnormalize(candidates.detach(), bounds=bounds)
    print(new_x.shape)
    new_obj_true = torch.stack([model.models[0](train_x).mean, model.models[1](train_x).mean], dim=-1).unsqueeze(1)
    #new_obj = new_obj_true + torch.randn_like(new_obj_true) * NOISE_SE
    return new_x, new_obj_true

In [36]:
from botorch.acquisition.monte_carlo import qNoisyExpectedImprovement


def optimize_qnparego_and_get_observation(model, train_x, train_obj, sampler):
    """Samples a set of random weights for each candidate in the batch, performs sequential greedy optimization
    of the qNParEGO acquisition function, and returns a new candidate and observation."""
    train_x = normalize(train_x, bounds)
    with torch.no_grad():
        pred = model.posterior(train_x).mean
        #pred = torch.concatenate([model.posterior(train_x[:,:,i]).mean for i, model in enumerate(model_qparego.models)],dim=-1)
    acq_func_list = []
    for _ in range(BATCH_SIZE):
        weights = sample_simplex(2, **tkwargs).squeeze()
        objective = GenericMCObjective(
            get_chebyshev_scalarization(weights=weights, Y=pred)
        )
        acq_func = qNoisyExpectedImprovement(  # pyre-ignore: [28]
            model=model,
            objective=objective,
            X_baseline=train_x.clone(),
            sampler=sampler,
            prune_baseline=False,
        )
        acq_func_list.append(acq_func)
    # optimize
    candidates, _ = optimize_acqf_list(
        acq_function_list=acq_func_list,
        bounds=bounds,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
    )
    # observe new values
    new_x = unnormalize(candidates.detach(), bounds=bounds)
    new_obj_true = model(new_x)
    #new_obj = new_obj_true + torch.randn_like(new_obj_true) * NOISE_SE
    return new_x, new_obj_true

In [37]:
import time
import warnings

from botorch import fit_gpytorch_mll
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.multi_objective.box_decompositions.dominated import (
    DominatedPartitioning,
)
from botorch.utils.multi_objective.pareto import is_non_dominated
torch.autograd.set_detect_anomaly(False)

warnings.filterwarnings("ignore", category=BadInitialCandidatesWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

N_BATCH = 20 if not SMOKE_TEST else 5
MC_SAMPLES = 128 if not SMOKE_TEST else 16

verbose = True

hvs_qparego, hvs_qehvi, hvs_qnehvi, hvs_random = [], [], [], []

# call helper functions to generate initial training data and initialize model
# train_x_qparego, train_obj_true_qparego, total_df = generate_initial_data(surrogates)
# mll_qparego, model_qparego = initialize_model(train_x_qparego, train_obj_true_qparego)

train_x_qehvi, train_obj_true_qehvi, total_df = generate_initial_data(surrogates)
# train_x_qnehvi, train_obj_true_qnehvi = (
#     train_x_qparego,
#     train_obj_true_qparego,
# )
# train_x_random, train_obj_true_random = (
#     train_x_qparego,
#     train_obj_true_qparego,
# )
mll_qehvi, model_qehvi = initialize_model(train_x_qehvi, train_obj_true_qehvi)
#mll_qnehvi, model_qnehvi = initialize_model(train_x_qnehvi, train_obj_true_qnehvi)

# compute hypervolume
bd = DominatedPartitioning(ref_point=ref_point, Y=train_obj_true_qehvi.detach().clone().squeeze(dim=1))
volume = bd.compute_hypervolume().item()

hvs_qparego.append(volume)
hvs_qehvi.append(volume)
hvs_qnehvi.append(volume)
#hvs_random.append(volume)

# run N_BATCH rounds of BayesOpt after the initial random batch
for iteration in range(1, N_BATCH + 1):

    t0 = time.monotonic()

    # fit the models
    # train_model(model_qparego, mll_qparego)
    model_qehvi.train()
    train_model(model_qehvi, mll_qehvi)
    model_qehvi.eval
    # train_model(model_qnehvi, mll_qnehvi)

    # define the qEI and qNEI acquisition modules using a QMC sampler
    # qparego_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))
    qehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))
    # qnehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))

    # optimize acquisition functions and get new observations
    # new_x_qparego, new_obj_true_qparego = optimize_qnparego_and_get_observation(
    #     model_qparego, train_x_qparego, train_obj_true_qparego, qparego_sampler
    # )
    new_x_qehvi, new_obj_true_qehvi = optimize_qehvi_and_get_observation(
        model_qehvi, train_x_qehvi, train_obj_true_qehvi, qehvi_sampler
    )
    # new_x_qnehvi, new_obj_true_qnehvi = optimize_qnehvi_and_get_observation(
    #     model_qnehvi, train_x_qnehvi, train_obj_true_qnehvi, qnehvi_sampler
    # )
    # new_x_random, new_obj_random, new_obj_true_random = generate_initial_data(
    #     n=BATCH_SIZE
    # )

    # update training points
    # train_x_qparego = torch.cat([train_x_qparego, new_x_qparego])
    # train_obj_true_qparego = torch.cat([train_obj_true_qparego, new_obj_true_qparego])

    train_x_qehvi = torch.cat([train_x_qehvi, new_x_qehvi])
    train_obj_true_qehvi = torch.cat([train_obj_true_qehvi, new_obj_true_qehvi], dim=0)

    # train_x_qnehvi = torch.cat([train_x_qnehvi, new_x_qnehvi])
    # train_obj_true_qnehvi = torch.cat([train_obj_true_qnehvi, new_obj_true_qnehvi])

    # train_x_random = torch.cat([train_x_random, new_x_random])
    # train_obj_true_random = torch.cat([train_obj_true_random, new_obj_true_random])

    # update progress
    for hvs_list, train_obj in zip(
        (hvs_random, hvs_qparego, hvs_qehvi, hvs_qnehvi),
        (
            #train_obj_true_random,
            # train_obj_true_qparego,
            train_obj_true_qehvi,
            # train_obj_true_qnehvi,
        ),
    ):
        # compute hypervolume
        bd = DominatedPartitioning(ref_point=ref_point, Y=train_obj.detach().clone().squeeze(dim=1))
        volume = bd.compute_hypervolume().item()
        hvs_list.append(volume)

    # reinitialize the models so they are ready for fitting on next iteration
    # Note: we find improved performance from not warm starting the model hyperparameters
    # using the hyperparameters from the previous iteration
    # mll_qparego, model_qparego = initialize_model(train_x_qparego, train_obj_qparego)
    mll_qehvi, model_qehvi = initialize_model(train_x_qehvi, train_obj_true_qehvi)
    # mll_qnehvi, model_qnehvi = initialize_model(train_x_qnehvi, train_obj_qnehvi)

    t1 = time.monotonic()

    if verbose:
        print(
            f"\nBatch {iteration:>2}: Hypervolume (random, qNParEGO, qEHVI, qNEHVI) = "
            f"({hvs_random[-1]:>4.2f}, {hvs_qparego[-1]:>4.2f}, {hvs_qehvi[-1]:>4.2f}, {hvs_qnehvi[-1]:>4.2f}), "
            f"time = {t1-t0:>4.2f}.",
            end="",
        )
    else:
        print(".", end="")

torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
-------------------
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([514, 104])
torch.Size([51

/Users/jingsk/miniconda3/envs/matminer-phaseshift/lib/python3.10/site-packages/botorch/acquisition/multi_objective/monte_carlo.py:203: NumericsWarning: qExpectedHypervolumeImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 qExpectedHypervolumeImprovement 	 --> 	 qLogExpectedHypervolumeImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)


torch.Size([4, 1, 104])
torch.Size([4, 1, 104])
torch.Size([2, 1, 104])
torch.Size([2, 1, 104])


RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [ ]:
model, train_x, train_obj, sampler = model_qehvi, train_x_qehvi, train_obj_true_qehvi, qparego_sampler

In [ ]:
train_obj_true_qehvi.shape

In [ ]:
init_X, init_y, _ = generate_initial_data(surrogates)

In [ ]:
init_y.shape

In [ ]:
train_obj_true_qehvi.shape

In [ ]:
bd = DominatedPartitioning(ref_point=ref_point, Y=init_y[:10,:,:].squeeze(dim=1))
volume = bd.compute_hypervolume().item(); volume

In [ ]:
bd = DominatedPartitioning(ref_point=ref_point, Y=train_obj_true_qehvi.squeeze(dim=1))
volume = bd.compute_hypervolume().item(); volume

In [ ]:
new_obj_true_qehvi.unsqueeze(1).shape

In [ ]:
model, train_x, train_obj, sampler = model_qparego, train_x_qparego, train_obj_true_qparego, qparego_sampler
"""Samples a set of random weights for each candidate in the batch, performs sequential greedy optimization
of the qNParEGO acquisition function, and returns a new candidate and observation."""
#train_x = normalize(train_x, bounds)
train_x = normalize(train_x, bounds)
with torch.no_grad():
    pred = model.posterior(train_x).mean
acq_func_list = []
for _ in range(BATCH_SIZE):
    weights = sample_simplex(2, **tkwargs).squeeze()
    objective = GenericMCObjective(
        get_chebyshev_scalarization(weights=weights, Y=pred)
    )
    acq_func = qNoisyExpectedImprovement(  # pyre-ignore: [28]
        model=model,
        objective=objective,
        X_baseline=train_x,
        sampler=sampler,
        prune_baseline=False,
    )
    acq_func_list.append(acq_func)
# optimize
candidates, _ = optimize_acqf_list(
    acq_function_list=acq_func_list,
    bounds=bounds,
    num_restarts=1,
    raw_samples=512,  # used for intialization heuristic
    options={"batch_limit": 5, "maxiter": 200},
)
# observe new values
new_x = unnormalize(candidates.detach(), bounds=bounds)
#new_obj_true = problem(new_x)
#new_obj = new_obj_true + torch.randn_like(new_obj_true) * NOISE_SE
#return new_x, new_obj_true

In [ ]:
normalize(train_x, bounds).shape

In [ ]:
candidates

In [ ]:
qehvi_sampler(model.posterior(normalize(train_x, bounds))).shape

In [ ]:
model, train_x, train_obj, sampler = model_qehvi, train_x_qehvi, train_obj_true_qehvi, qehvi_sampler
with torch.no_grad():
    pred = model.posterior(normalize(train_x, bounds)).mean
    #pred = torch.concatenate([model.posterior(train_x[:,:,i]).mean for i, model in enumerate(model_qparego.models)],dim=-1)
partitioning = FastNondominatedPartitioning(
    ref_point=ref_point,
    Y=pred,
)
acq_func = qExpectedHypervolumeImprovement(
    model=model,
    ref_point=ref_point,
    partitioning=partitioning,
    sampler=sampler,
)
# optimize
candidates, _ = optimize_acqf(
    acq_function=acq_func,
    bounds=bounds,
    q=BATCH_SIZE,
    num_restarts=NUM_RESTARTS,
    raw_samples=RAW_SAMPLES,  # used for intialization heuristic
    options={"batch_limit": 5, "maxiter": 200},
    sequential=True,
)
# observe new values
new_x = unnormalize(candidates.detach(), bounds=bounds)
print(new_x.shape)
new_obj_true = torch.stack([model.models[0](new_x).mean, model.models[1](new_x).mean], dim=-1).unsqueeze(1)

In [ ]:
bounds.shape

In [ ]:
optimize_acqf(
    acq_function=acq_func,
    bounds=bounds,
    q=BATCH_SIZE,
    num_restarts=NUM_RESTARTS,
    raw_samples=RAW_SAMPLES,  # used for intialization heuristic
    options={"batch_limit": 5, "maxiter": 200},
    sequential=True,
)

In [ ]:
model.models[0].to_scale

In [ ]:
model.num_outputs

In [ ]:
model.posterior(train_x[:,:,0]).mean

In [ ]:
model.posterior(torch.concatenate([train_x[:,:,0],train_x[:,:,1]],dim=0)).mean

In [ ]:
train_x_qparego.shape

In [ ]:
model_qparego.posterior(train_x_qparego[:,:,0])

In [ ]:
model_qparego.models[0].posterior(train_x_qparego[:,:,0]).mean.shape

In [ ]:
torch.concatenate([model.posterior(train_x_qparego[:,:,i]).mean for i, model in enumerate(model_qparego.models)],dim=-1).shape

In [ ]:
torch.mean(, dim=1)

In [ ]:
model_qparego.models[0]

In [ ]:
train_x_qparego.shape

In [ ]:
train_obj_true_qparego.shape

In [ ]:
bd = DominatedPartitioning(ref_point=ref_point, Y=train_obj_true_qehvi.squeeze(dim=1))
volume = bd.compute_hypervolume().item()

In [ ]:
train_x_qehvi.shape

In [ ]:
train_obj_true_qehvi.shape

In [ ]:
model_qehvi.train_inputs[0][0].shape

training_set = [torch.tensor(X[0], **tkwargs) for X in model_qehvi.train_inputs]
torch.stack(training_set, dim=-1).shape


In [ ]:
model_qehvi.eval()
model_qehvi(train_x_qehvi)

In [ ]:
def generate_initial_data(surrogates):
    from utils.model import evaluateGP
    X = np.concatenate([surrogate.X for surrogate in surrogates],axis=0)
    y = [evaluateGP(surrogate.model, X) for surrogate in surrogates][0]
    X = torch.tensor(X, **tkwargs)
    y = [torch.tensor(array, **tkwargs) for array in y]
    X = torch.stack((X, X),dim=-1)
    y = torch.stack(y,dim=-1)
    
    return X, y

In [ ]:
from utils.model import evaluateGP
X = []
X = np.concatenate([surrogate.X for surrogate in surrogates],axis=0)
y = [evaluateGP(surrogate.model, X) for surrogate in surrogates][0]

In [ ]:
a,b = generate_initial_data(surrogates)

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
X.shape

In [ ]:
y.sh